# Wrangle and Analyze Data

## Introduction

The dataset that is used for this exercise is an extended archive of tweets from the [WeRateDogs](https://en.wikipedia.org/wiki/WeRateDogs) twitter account. 

### Project Details

The dataset consists of three files:

1. An enhanced tweet archive provided by Udacity and downloaded from its portal.

2. Tweet Image Predictions which come in another fle provided by Udacity, but which is downloaded via server call using Python's requests library.

3. Retweet and favorite counts for each of the tweets in the enhanced tweet archive which I will gather via the Twitter API using the Tweepy library. This data is then saved to a csv file locally which will be submitted to Udacity in fulfillment of the project requirements.

Once these files are gathered, I will assess each of them for both quality and for tidiness.  After any quality issues are fixed and the dataframes saved in a tidy format, we will have a file or files that will enable analyis.  We will produce at least one visualization which will enable us to summarize our findings.

## Load Python Libraries for Gathering, Assesment and Cleaning

Load packages and libraries required for subsequent blocks.

In [2]:
# import packages
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os
from timeit import default_timer as timer

## Gather Data

Gather data need for analyzing We Rate Dogs (@dog_rates) twitter feed. There are three files comprising the broader data set.

### File on-hand: Twitter Archive

This file was presented on and downloaded from the Udacity website. So it is simply read into the dataframe using the Pandas 'read_csv' function.

In [3]:
# read dog_rates twitter archive
df = pd.read_csv('twitter-archive-enhanced.csv')

In [4]:
# printing a sample of the dataframe allows for visual inspection.
df.sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1922,674255168825880576,NaN,NaN,2015-12-08 15:52:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Bobbay. He's a marshmallow wizard...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674255168...,10,10,Bobbay,None,None,None,None
147,863432100342583297,NaN,NaN,2017-05-13 16:33:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Belle. She's never been more pupset. E...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/863432100...,12,10,Belle,None,None,None,None
1684,681679526984871937,NaN,NaN,2015-12-29 03:33:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Can you spot Toby the guilty pupper? 7/10 woul...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/681679526...,7,10,None,None,None,pupper,None


In [5]:
# using the pandas info() function shows all the variables, their data-types and gives us 
# a count of the entries and missing values.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

### Download File from Server: Tweet Image Predictions

For the tweet image predictions file, we download the file from a Udacity server using the 'requests' library. This file is then written (saved) to the project directory.

In [6]:
# download tweet image predictions from server - save into workspace
image_prediction = {'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'}
# 'for' loop downloads all files at URL list in ebert_review_urls variable
for url in image_prediction:
    response = requests.get(url)
    with open(os.path.join(url.split('/')[-1]), mode = 'wb') as file:
        file.write(response.content)

We can verify that the download occurred by viewing the project directory to confirm there is an 'image_predictions.csv' file in it

In [7]:
os.listdir()

['stuff.txt',
 'details_t2_p3.txt',
 'wrangle_act.ipynb',
 'twitter-archive-enhanced.csv',
 '.DS_Store',
 'README.md',
 'project_intro_t2p3',
 '.gitignore',
 'tweepy_practice.py',
 '.ipynb_checkpoints',
 'image-predictions.tsv',
 '.git',
 'rubric_t2p3.txt',
 'tweet_json.txt']

We then open the file using the 'read_csv' function with the tab-separator indicated.

In [5]:
df2 = pd.read_csv('image-predictions.tsv',sep='\t')

Next, we perform a brief visual inspection and examine the fields using the info() function, same as with the tweet archive file.

In [10]:
df2.sample(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
79,667453023279554560,https://pbs.twimg.com/media/CUNE_OSUwAAdHhX.jpg,1,Labrador_retriever,0.825670,True,French_bulldog,0.056639,True,Staffordshire_bullterrier,0.054018,True
1312,754482103782404096,https://pbs.twimg.com/ext_tw_video_thumb/75448...,1,tub,0.596796,False,bathtub,0.381098,False,shower_curtain,0.017629,False
1929,858471635011153920,https://pbs.twimg.com/media/C-nnZBdXkAAB-wg.jpg,1,Pembroke,0.987407,True,Cardigan,0.008723,True,basenji,0.003424,True


In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


### Create File Dynamically Using Tweepy to Query Twitter API: tweet_json.txt File

The third file needed for this project containing information such as 'retweet' and 'favorite' counts is created dynamically by querying the Twitter api.

First steps took place elsewhere. At apps.twitter.com, I created a (dummy) app which enabled me to create authorization keys as required by Twitter.

Second step is to enter code as indicated in the [Tweepy documentation](http://tweepy.readthedocs.io/en/v3.6.0/getting_started.html#introduction) authenticating access to the API.  These are omitted from the project submission per security best practices and as specified in the project rubric.

In [82]:
# Tweepy auth stuff
# place keys here - delete manually for now

In [110]:
# Tweepy auth stuff continued: note the parameters included in the API call
# setting parser to tweepy.JSONParser ensures that it returns good JSON
# rate_limit parameters enable the function to work through the Twitter rate limit
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True,\
                 wait_on_rate_limit_notify=True)

The next step is for Tweepy to access twitter data for the 2356 tweets in the tweet_archive file and to download that data as a json file. First we take the tweet_id variable in the first dataframe (created from the twitter archive csv) and turn it into a list.  The length of that list is returned as a test (2356 is the correct number).

In [87]:
# put all of the tweetids from the twitter_archive csv/dataframe into a list
tweetids = df['tweet_id'].tolist()
# tweetids
len(tweetids)

2356

Then we are going to loop through the list of tweet_ids to access the corresponding records via the Twitter api. We are using a try-except block to save the twitter status for the ids that have records and, separately, to keep a file of the ids which have been deleted. Thanks to the "wait_on_rate_limit" parameter in the api call, the function will not time out while waiting for data from the 'rate limited' API.

In [89]:
# List of dictionaries to build and convert to a DataFrame later
tweet_list = []
tweet_errors = []
start = timer()
for tweet in tweetids:
    try:
        full_status = api.get_status(tweet, tweet_mode='extended')
        tweet_list.append(full_status)
        end = timer()
    except Exception as e:
        end = timer()
        print('No status found for tweetid ' + str(tweet))
        print('Time elapsed to error ' + str(end - start))
        tweet_errors.append(tweet)
print('Total runtime for function ' + str((end - start)/60) + ' minutes.')
print(len(tweet_list))
print (len(tweet_errors))

No status found for tweetid 888202515573088257
Time elapsed to error 4.967670181998983
No status found for tweetid 873697596434513921
Time elapsed to error 21.60596933600027
No status found for tweetid 869988702071779329
Time elapsed to error 26.490966256998945
No status found for tweetid 866816280283807744
Time elapsed to error 29.46525864698924
No status found for tweetid 861769973181624320
Time elapsed to error 34.22458414500579
No status found for tweetid 845459076796616705
Time elapsed to error 53.73645277399919
No status found for tweetid 842892208864923648
Time elapsed to error 56.511891781992745
No status found for tweetid 837012587749474308
Time elapsed to error 65.38332627300406
No status found for tweetid 827228250799742977
Time elapsed to error 83.12681407199125
No status found for tweetid 802247111496568832
Time elapsed to error 122.45716445101425
No status found for tweetid 775096608509886464
Time elapsed to error 169.60564937099116


Rate limit reached. Sleeping for: 468
Rate limit reached. Sleeping for: 708


Time total runtime for function 2.826760822849853 minutes.
2345
11


There were 2345 tweets that had meta-data returned and 11 which had been deleted subsequent to the creation of the twitter_archive csv.  Next let's take a look at one line from the tweet_list to see what the data looks like.

In [90]:
tweet_list[:1]

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
  'display_text_range': [0, 85],
  'entities': {'hashtags': [],
   'media': [{'display_url': 'pic.twitter.com/MgUWQ76dJU',
     'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
     'id': 892420639486877696,
     'id_str': '892420639486877696',
     'indices': [86, 109],
     'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
     'sizes': {'large': {'h': 528, 'resize': 'fit', 'w': 540},
      'medium': {'h': 528, 'resize': 'fit', 'w': 540},
      'small': {'h': 528, 'resize': 'fit', 'w': 540},
      'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
     'type': 'photo',
     'url': 'https://t.co/MgUWQ76dJU'}],
   'symbols': [],
   'urls': [],
   'user_mentions': []},
  'extended_entities': {'media': [{'display_url': 'pic.twitter.com/MgUWQ76dJU',
     'expanded_url': 'htt

This appears to be well-formed JSON: we'll know for sure when we save it and try to reopen that file.  Next step is to store the output as a text file 'tweet_json.txt' using the json.dump function.

In [91]:
# Store tweet info in JSON format

with open('tweet_json.txt', 'w') as outfile:  
    json.dump(tweet_list, outfile)

Our test to make sure it worked is to see if there is a 'tweet_json.txt' file in our working directory.

In [12]:
os.listdir()

['stuff.txt',
 'details_t2_p3.txt',
 'wrangle_act.ipynb',
 'twitter-archive-enhanced.csv',
 '.DS_Store',
 'README.md',
 'project_intro_t2p3',
 '.gitignore',
 'tweepy_practice.py',
 '.ipynb_checkpoints',
 'image-predictions.tsv',
 '.git',
 'rubric_t2p3.txt',
 'tweet_json.txt']

And it is there! Our next step is to read this text file into a dataframe using Pandas read_json function

In [9]:
df_json = pd.read_json(open('tweet_json.txt'))

In [10]:
df_json.head(1)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
0,NaN,NaN,2017-08-01 16:23:56,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...",38693,False,This is Phineas. He's a mystical boy. Only eve...,NaN,...,NaN,NaN,NaN,NaN,8559,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."


This is a massive file with a ton of data, so I only printed out the first item to make sure that the dataframe is properly formed. The next (and *final*) step in gathering our data is to select only the variables needed for this project ('id','retweet_count','favorite_count') and put them into a new dataframe.

In [11]:
df3 = df_json[['id','retweet_count','favorite_count']]

In [12]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2345 entries, 0 to 2344
Data columns (total 3 columns):
id                2345 non-null int64
retweet_count     2345 non-null int64
favorite_count    2345 non-null int64
dtypes: int64(3)
memory usage: 73.3 KB


In [13]:
df3.head(3)

,id,retweet_count,favorite_count
0,892420643555336193,8559,38693
1,892177421306343426,6294,33167
2,891815181378084864,4176,24967


We can see that the dataframe for these items is complete and tidy.  Now on to further assessment of the data we have gathered in these three files.

## Assess Data

For each of the three dataframes created from the three files, we will perform the following assessment steps:

1. Review df.info() results in order to:
  - Count valid and null values in each variable
  - Review and check variable names
  - Validate data types
2. Print a sample of 10 values from the dataframe and perform a visual inspection for quality and validity.
3. Run a duplicates function to check for duplicated rows (another quality/validity check).
4. Run value counts function to check quality and validity of entries.

**Please note that the issues and potential-issues will be tracked below in the Notes section. And we will only correct a smaller subset of these items per the project instructions.**

### WeRateDogs Tweet Archive from File On-Hand

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

Variable `timestamp` is string, probably should be a datetime.

In [18]:
df.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
735,781163403222056960,NaN,NaN,2016-09-28 16:07:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...","We normally don't rate lobsters, but this one ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/781163403...,10,10,None,None,None,None,None
1743,679405845277462528,NaN,NaN,2015-12-22 20:59:10 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Crazy unseen footage from Jurassic Park. 10/10...,NaN,NaN,NaN,https://vine.co/v/iKVFEigMLxP,10,10,None,None,None,None,None
2107,670465786746662913,NaN,NaN,2015-11-28 04:54:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Silly dog here. Wearing bunny ears. Nice long ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670465786...,7,10,None,None,None,None,None
152,862722525377298433,NaN,NaN,2017-05-11 17:34:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Dave. He passed the h*ck out. It's bar...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/862722525...,11,10,Dave,None,None,None,None
1274,709198395643068416,NaN,NaN,2016-03-14 02:04:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...","From left to right:\nCletus, Jerome, Alejandro...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/709198395...,45,50,None,None,None,None,None
1548,689275259254616065,NaN,NaN,2016-01-19 02:36:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Lucky. He was showing his friends an extr...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/689275259...,10,10,Lucky,None,None,None,None
1094,736736130620620800,NaN,NaN,2016-05-29 01:49:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Chadrick. He's gnarly af 13/10 https:/...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/736736130...,13,10,Chadrick,None,None,None,None
2028,671866342182637568,NaN,NaN,2015-12-02 01:39:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Dylan. He can use a fork but clearly can'...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671866342...,10,10,Dylan,None,None,None,None
1791,677335745548390400,NaN,NaN,2015-12-17 03:53:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Downright inspiring 12/10 https://t.co/vSLtYBWHcQ,NaN,NaN,NaN,https://vine.co/v/hbLbH77Ar67,12,10,None,None,None,None,None
1737,679530280114372609,NaN,NaN,2015-12-23 05:13:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Guys this really needs to stop. We've been ove...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/679530280...,7,10,a,None,None,None,None


In [20]:
# checking for duplicate rows
df[df.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [24]:
# checking for duplicated tweet id
df[df['tweet_id'].duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [32]:
# run value_counts on 'name' variable
vc = df['name'].value_counts()
print(vc[vc > 3].to_string())

None        745
a            55
Charlie      12
Cooper       11
Oliver       11
Lucy         11
Lola         10
Tucker       10
Penny        10
Winston       9
Bo            9
Sadie         8
the           8
Daisy         7
an            7
Bailey        7
Buddy         7
Toby          7
Leo           6
Scout         6
Dave          6
Oscar         6
Milo          6
Rusty         6
Bella         6
Koda          6
Jax           6
Stanley       6
Jack          6
George        5
Sammy         5
Bentley       5
Louis         5
Larry         5
Gus           5
Finn          5
Chester       5
Alfie         5
very          5
Phil          5
Sunny         5
Oakley        5
Jerry         4
Derek         4
Jeffrey       4
Reggie        4
Archie        4
Maggie        4
Sampson       4
Brody         4
Beau          4
Duke          4
Chip          4
quite         4
Luna          4
Clarence      4
Moose         4
Maddie        4
Riley         4
Walter        4
Cassie        4
Scooter       4
Hank    

In [33]:
df['doggo'].value_counts()

None     2259
doggo      97
Name: doggo, dtype: int64

In [34]:
df['floofer'].value_counts()

None       2346
floofer      10
Name: floofer, dtype: int64

In [35]:
df['puppo'].value_counts()

None     2326
puppo      30
Name: puppo, dtype: int64

In [36]:
df['pupper'].value_counts()

None      2099
pupper     257
Name: pupper, dtype: int64

### Image Prediction Table (df2) Downloaded from Server

In [131]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


The datatypes look correct here.

In [6]:
df2.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
269,670815497391357952,https://pbs.twimg.com/media/CU83IZ8W4AEIh4y.jpg,1,American_Staffordshire_terrier,0.919714,True,Staffordshire_bullterrier,0.073430,True,bull_mastiff,0.000906,True
1548,792883833364439040,https://pbs.twimg.com/media/CwDjoH3WAAIniIs.jpg,3,jack-o'-lantern,0.999306,False,basketball,0.000113,False,standard_poodle,0.000083,True
1159,733828123016450049,https://pbs.twimg.com/media/Ci8UxxcW0AYgHDh.jpg,2,beagle,0.472324,True,Walker_hound,0.121779,True,Saint_Bernard,0.114640,True
1076,717421804990701568,https://pbs.twimg.com/media/CfTLUYWXEAEkyES.jpg,2,miniature_pinscher,0.286479,True,Italian_greyhound,0.084134,True,beagle,0.064697,True
672,683142553609318400,https://pbs.twimg.com/media/CXsChyjW8AQJ16C.jpg,1,Leonberg,0.605851,True,chow,0.183470,True,German_shepherd,0.079662,True
449,674737130913071104,https://pbs.twimg.com/media/CV0l10AU8AAfg-a.jpg,1,Pomeranian,0.948537,True,schipperke,0.014310,True,Chihuahua,0.008120,True
1158,733822306246479872,https://pbs.twimg.com/media/Ci8Pfg_UUAA2m9i.jpg,1,Lhasa,0.457356,True,Shih-Tzu,0.371282,True,Tibetan_terrier,0.048359,True
1406,770069151037685760,https://pbs.twimg.com/media/Cq_Vy9KWcAIUIuv.jpg,1,Boston_bull,0.414965,True,American_Staffordshire_terrier,0.286985,True,Staffordshire_bullterrier,0.114970,True
1858,841833993020538882,https://pbs.twimg.com/ext_tw_video_thumb/81742...,1,ice_bear,0.336200,False,Samoyed,0.201358,True,Eskimo_dog,0.186789,True
1231,745789745784041472,https://pbs.twimg.com/media/ClmT0KHWkAAXbhy.jpg,1,Pekinese,0.984267,True,Shih-Tzu,0.008942,True,cocker_spaniel,0.001928,True


In [7]:
df2[df2['tweet_id'].duplicated()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


So no duplicates in that file.

### Retweet and Favorite Counts Gathered from Twitter API

In [137]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2345 entries, 0 to 2344
Data columns (total 3 columns):
id                2345 non-null int64
retweet_count     2345 non-null int64
favorite_count    2345 non-null int64
dtypes: int64(3)
memory usage: 73.3 KB


Data types here are correct - all are integers. Also worth noting: 2345 items = 2356 in the tweet archive - 11 tweets that were deleted subsequently (noted above after we queried the archive.

In [14]:
df3.sample(10)

,id,retweet_count,favorite_count
804,771004394259247104,246,0
2093,670668383499735048,5321,11113
27,886680336477933568,4489,22367
1507,690932576555528194,1116,3513
1916,674051556661161984,509,1507
1770,677698403548192770,347,1293
1154,722974582966214656,1711,4375
594,798585098161549313,6431,0
48,882762694511734784,4945,28261
190,855138241867124737,49,0


In [16]:
df3[df3['id'].duplicated()]

,id,retweet_count,favorite_count


No duplicate ids. Worth noting that the `id` field here is called `tweet_id` in the other two dataframes: this might be worth changing to enhance readability and clarity.

# Notes

## Quality

### Tweet Archive Table (df) from "File On-Hand"

- Timestamp variable is a string: should be converted to a datetime format.
- 'Name'variable includes 55 "the", 8 "a", 7 "an", 5 "very", 4 "just" "quite" and "one."  Not to mention 745 with no name.
- Retweets need to be identified and removed.
- 11 Tweets that have been subsequently deleted and so are missing retweet and favorite numbers can be removed
- Explain delta between this table which includes 2356 records and the image prediction table (df2) containing 2057 records.
- Examine tweets containing "in_reply_to" fields and determine whether they are valid for analysis.
- Determine whether missing values (NaN) need to be imputed or otherwise replaced.  Hypothesis here is they do not, but that may prove untrue.
- Need to examine following columns for potential quality issues (mentioned in the "Project Motivation" file):

   1. Dog names
   1. Ratings
   1. Dog "Stages" - Pupper, Floofer, etc.

### Image Prediction Table (df2) Downloaded from Server

- Identify delta between df2 with 2057 records and df1 with 2356 records (see above)

### Retweet and Favorite Counts Gathered from Twitter API (df3)

- Figure out why certain tweets have thousands of retweets and 0 favorites: is there a problem with the quality of the data or something else (question to self: when were favorites introduced on twitter??)
- `id` field in df3 is called `tweet_id` in the other two dataframes: marginally worth changing.

# Tidiness Issues

- Since all three of these tables concern the same set of observations (observation = tweetid), the tidiest version of this data will be a single table. This will enable us to eliminate duplicate tweet ids and some other columns (like the photo url in df2). Accordingly we will join the three tables and will remove duplicate columns.
- Arguably, for ease of performing statistical calculations and creating visualizations, we can further trim the consolidated dataframe by eliminating extranaeous variables.
- Dog stage columns need to be melted into one single "Dog Stage" column with the values either "doggo", "floofer", "pupper" or "poppo" or "none"
- Note to self: it's possible that in order to do this properly we will need to convert the "None" string into NaN missing value. That would be a quality issue in df table above.



## Preliminary Cleaning Work

In [139]:
# returning all columns that are retweets - these need to be eliminated from final dataframe
df[df['retweeted_status_id'].notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
74,878316110768087041,NaN,NaN,2017-06-23 18:17:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Terrance. He's being yelle...,6.690004e+17,4.196984e+09,2015-11-24 03:51:38 +0000,https://twitter.com/dog_rates/status/669000397...,11,10,Terrance,None,None,None,None
78,877611172832227328,NaN,NaN,2017-06-21 19:36:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @rachel2195: @dog_rates the boyfriend and h...,8.768508e+17,5.128045e+08,2017-06-19 17:14:49 +0000,https://twitter.com/rachel2195/status/87685077...,14,10,None,None,None,pupper,None
91,874434818259525634,NaN,NaN,2017-06-13 01:14:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Coco. At first I though...,8.663350e+17,4.196984e+09,2017-05-21 16:48:45 +0000,https://twitter.com/dog_rates/status/866334964...,12,10,Coco,None,None,None,None
95,873697596434513921,NaN,NaN,2017-06-11 00:25:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Walter. He won't start ...,8.688804e+17,4.196984e+09,2017-05-28 17:23:24 +0000,https://twitter.com/dog_rates/status/868880397...,14,10,Walter,None,None,None,None
97,873337748698140672,NaN,NaN,2017-06-10 00:35:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Sierra. She's one preci...,8.732138e+17,4.196984e+09,2017-06-09 16:22:42 +0000,https://www.gofundme.com/help-my-baby-sierra-g...,12,10,Sierra,None,None,pupper,None
